1) Como framework de cluster computing, Apache Spark provee una abstracción llamada RDDs (Resilent Distributed Datasets). De los siguientes opciones, solo una es correcta en relación los RDDs.	 
 		 a) Son la principal abstracción de datos usada en Hadoop.
         b) Son particularmente lentos para tareas de Interactive Mining por las operaciones de entrada-salida que requieren.
         c) Son reconstruidos automáticamente por Spark frente a un fallo en una maquina del cluster o si un cierto job se encuentra funcionando lento.
         d) Frente a un fallo, pueden quedar en estado inconsistente.
         
RTA: la opción elegida es la opción c).

2) Dado el siguiente código de pyspark e ignorando la definicion de pairRDD -- print pairRDD.groupByKey().mapValues(lambda x: sum(x)).collect() print pairRDD.reduceByKey(add).collect() -- Obtenemos la siguiente salida -- [('a', 3), ('b', 1)] [('a', 3), ('b', 1)] -- ¿Como debería estar definida pairRDD para obtener ese resultado?

a) 	pairRDD = [('a', 1), ('a', 1), ('a', 1), ('b', 1)]
b) 	pairRDD = sc.parallelize([('a', 2), ('a', 1), ('b', 1)])
c) 	pairRDD = sc.parallelize([('b', 1), ('b', 1), ('b', 1), ('a', 1)]).repartition(2)
d) 	pairRDD = sc.parallelize([('a', 1), ('a', 2), ('b', 2)])

RTA: la opción elegida es la opción b).

In [61]:
import pyspark
import csv

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [162]:
# Pruebo con el rdd del punto a).
pairRDD = [('a', 1), ('a', 1), ('a', 1), ('b', 1)]
print pairRDD.groupByKey().mapValues(lambda x: sum(x)).collect() 
print pairRDD.reduceByKey(add).collect()

[('a', 1), ('a', 1), ('a', 1), ('b', 1)]

In [238]:
# Pruebo con el rdd del punto b).
pairRDD = sc.parallelize([('a', 2), ('a', 1), ('b', 1)])
print pairRDD.groupByKey().mapValues(lambda x: sum(x)).collect() 
print pairRDD.reduceByKey(lambda x, y: x + y).collect()
pairRDD.collect()

[('a', 3), ('b', 1)]
[('a', 3), ('b', 1)]


[('a', 2), ('a', 1), ('b', 1)]

In [18]:
# Pruebo con el rdd del punto c).
pairRDD = sc.parallelize([('b', 1), ('b', 1), ('b', 1), ('a', 1)]).repartition(2)
print pairRDD.groupByKey().mapValues(lambda x: sum(x)).collect() 
print pairRDD.reduceByKey(lambda x, y: x + y).collect()

[('a', 1), ('b', 3)]
[('a', 1), ('b', 3)]


In [32]:
# Pruebo con el rdd del punto d).
pairRDD = sc.parallelize([('a', 1), ('a', 2), ('b', 2)])
print pairRDD.groupByKey().mapValues(lambda x: sum(x)).collect() 
print pairRDD.reduceByKey(lambda x, y: x + y).collect()

[('a', 3), ('b', 2)]
[('a', 3), ('b', 2)]


3) Tengo un RDD con los siguientes campos: padron, apellido_nombre, fecha_nacimiento, fecha_ingreso, codigo_carrera.

Indicar cuales de los siguientes es correcto:

a) 	rdd.filter(lambda row: row.codigo_carrera == '10').filter(lambda row: row.fecha_ingreso > '2016-01-01').takeSample(false, 10) obtiene la cantidad de alumnos inscriptos en la carrera 10 a partir del 2016.
b) 	rdd.filter(lambda row: row.codigo_carrera == '10').filter(lambda row: row.fecha_ingreso > '2016-01-01').map(lambda row: (len(row.apellido_nombre), 1).reduceByKey(lambda a, b: a+b).collect() obtiene la cantidad de alumnos por longitud de apellido y nombre para los alumnos de la carrera 10.
c) 	rdd.filter(lambda row: row.codigo_carrera == '10').filter(lambda row: row.fecha_ingreso > '2016-01-01').map(lambda row: (len(row.apellido_nombre), 1).reduceByKey(lambda a, b: a+b).collect() obtiene los apellidos de los alumnos que hayan ingresado despues del 2016-01-01 a la carrera 10.
 d) 	rdd.map(lambda row: (row.fecha_ingreso.year, 1)).reduceByKey(lambda a, b: a+b).collect() obtiene la cantidad de inscriptos por año.

# Analizo el punto a).
## Obtiene la cantidad de alumnos inscriptos en la carrera 10 a partir del 2016.

### Con este filter me quedo con los alumnos de la carrera 10.
rdd.filter(lambda row: row[3] == "10")

### En este filter me quedo con los alumnos anotados a partiro del dia 2016-01-02.
.filter(lambda row: row.fecha_ingreso > '2016-01-01')

## RTA: FALSO - No contempla los alumnos anotados el día 2016-01-01 que también forma parte del año 2016.

# Analizo el punto b).
## Obtiene la cantidad de alumnos por longitud de apellido y nombre para los alumnos de la carrera 10.
### Con este filter me quedo con los alumnos de la carrera 10.
rdd.filter(lambda row: row.codigo_carrera == '10')

### En este filter me quedo con los alumnos anotados a partiro del dia 2016-01-02.
.filter(lambda row: row.fecha_ingreso > '2016-01-01')

### Este map genera una tupla con la longitud de campo "apellido_nombre" y un 1: (longitud, 1).
.map(lambda row: (len(row.apellido_nombre), 1)

### Hace un reduce por longitud y suma la cantidad de veces que aparece la misma longitud: (longitud, veces)
.reduceByKey(lambda a, b: a+b).collect() 

## RTA: FALSO - No se consideran los alumnos totales, sino aquellos que ingresaron a partir del día 2016-01-02.
                                                                                                            

# Analizo el punto c).
## Obtiene los apellidos de los alumnos que hayan ingresado despues del 2016-01-01 a la carrera 10.

### Es la misma instrucción que en el punto b).

## RTA: Verdadero.

# Analizo el punto d).
## Obtiene la cantidad de inscriptos por año.

### Esta instrucción falla ya que no existe row.fecha_ingreso.year (En ningún momento la crea.)
rdd.map(lambda row: (row.fecha_ingreso.year, 1))
.reduceByKey(lambda a, b: a+b)
.collect()

## RTA: FALSA.

# ESTABA MAL LA RESPUESTA!

4) Tenemos un RDD con números enteros llamado intergerListRDD al cual le aplicamos las siguientes acciones. -- integersListRDD.reduce(lambda a, b: a - b) integersListRDD.repartition(6).reduce(lambda a, b: a - b) -- Que podemos decir los resultados de la aplicar el reduce por un lado y de aplicar del repartition y el reduce.	 
 		 a) 	Los resultados serán iguales dado que no hay que tener consideraciones especiales al aplicar una operacion de reduce sobre los datos.
 		 b) 	Los resultados serán distintos, dado que el reduce es aplicado a nivel de partición y luego nuevamente para agregar resultados de particiones, por lo que se requiere que la función planteada de sea conmutativa y asociativa
 		 c) 	Los resultados serán distintos dado que la operacion de repartition crea un nuevo RDD con el exacto número de particiones indicado (en este caso 6).
 		 d) 	Los resultados seran iguales dado que la operacion de resta es no conmutativa y asociativa.

In [192]:
integersListRDD = sc.parallelize([4325543, 32452345, 98567456, 67856, 34523, 3845, 56867, 2134768, 4366, 34435, 546756, 567546, 5674567, 89780, 80768965, 34534, 896789, 56678, 456435, 678567, 56745642, 324234, 678567, 4356, 4564, 435645, 4356435, 4563456, 456768, 32452345, 123143, 786554, 324523, 69787, 32423, 68976, 3252, 768567, 325321, 5674564334, 3245234, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 34345, 234 ,467, 345, 435, 567, 234, 768, 634, 58543, 3245, 3453, 345])
print integersListRDD.reduce(lambda a, b: a - b)
print integersListRDD.repartition(6).reduce(lambda a, b: a - b)

5563370559
5987639305


# Analizo las posibles respuestas

## a) Los resultados serán iguales dado que no hay que tener consideraciones especiales al aplicar una operacion de reduce sobre los datos.
FALSA - Las funciones que se aplican mediante el reduce deben ser conmutativas y asociativas. En este caso si los datos estan todos en la misma particion difiere de si los datos estan distribuidos en distintas particiones, por lo que la operación de resta no va a dar lo mismo.

## b) Los resultados serán distintos, dado que el reduce es aplicado a nivel de partición y luego nuevamente para agregar resultados de particiones, por lo que se requiere que la función planteada de sea conmutativa y asociativa.
VERDADERA.

## c) Los resultados serán distintos dado que la operacion de repartition crea un nuevo RDD con el exacto número de particiones indicado (en este caso 6).
FALSA - No se crean nuevos RDDs, simplemente se distribuye sus datos en diferentes particiones.

## d) Los resultados seran iguales dado que la operacion de resta es no conmutativa y asociativa.
FALSA - Para que el reduce dé resultados iguales, la operación a aplicar debe ser conmutativa y asociativa.

5) Archivo.txt:

datos spark guia

spark hadoop 

guia datos spark gradience

Corremos:

sc.textFile("Archivo.txt")
.flatMap(lambda a: a.split())
.map(lambda a: (a, 1))
.reduceByKey(lambda a, b: a+b)
.takeOrdered(3, key = lambda x: -x[1])

Cual de las respuestas es correcta?
 
 		 a) 	El flatMap devuelve un registro para cada palabra en el archivo.
 		 b) 	El takeOrdered obtiene las 3 primeras palabras ordenadas alfabeticamente.
 		 c) 	Map no puede funcionar desde un archivo.
 		 d) 	No es posible hacer un takeOrdered porque hay un reduceByKey antes.

In [232]:
archivo = sc.textFile("Data/Archivo.txt")

# Creo un registro por cada palabra.
archivo = archivo.flatMap(lambda a: a.split())

# Por cada palabra creo un registro que sea una tupla del estilo: (palabra, 1)
archivo = archivo.map(lambda a: (a, 1))

# Con el reduce obtengo un registro por palabra con la suma de sus apariciones: (palabra, suma)
archivo = archivo.reduceByKey(lambda a, b: a + b)

# El take orderen me devuelve los primeros 3 registros cuyo valor de la suma sea mayor.
archivo.takeOrdered(3, key = lambda x: -x[1])

[(u'spark', 3), (u'datos', 2), (u'guia', 2)]

# Analizo las opciones:
## a) El flatMap devuelve un registro para cada palabra en el archivo.
FALSO - El flatMap aplica la función a todos los elementos del RDD y luego hace un flat de los resultdos.
## b) El takeOrdered obtiene las 3 primeras palabras ordenadas alfabeticamente.
FALSO - Obtiene las 3 palabras con mayor frecuencia.
## c) Map no puede funcionar desde un archivo.
VERDADERO - Seleccionada por descarte. No entendí bien a que se refería.
## d) No es posible hacer un takeOrdered porque hay un reduceByKey antes.
FALSO - El take ordered se puede hacer sobre cualquier rdd sobre el cual tengamos alguna propiedad para comparar.

# ESTABA MAL LA RESPUESTA! ERA LA a).

6) Archivo.txt:

datos spark guia

spark hadoop 

guia datos spark gradience

Corremos:

sc.textFile("Archivo.txt")
.flatMap(lambda a: a.split())
.map(lambda a: (a, 1))
.reduceByKey(lambda a, b: a+b)
.takeOrdered(3, key = lambda x: -x[1])

Cual de las respuestas es correcta?
 
 		 a) 	El resultado solo tiene 3 registros.
 		 b) 	El resultado es la tercer palabra que más apariciones tiene.
 		 c) 	El resultado son las 3 palabras más largas.
 		 d) 	Con flatMap obtengo un registro para cada letra del archivo.

# Analizo las opciones.
La instrucción es la misma que en el punto 5).

## a) El resultado solo tiene 3 registros.
VERDADERO - En el take ordered le pasamos el parametro 3 para que nos devuelva las 3 palabras con mas frecuencia.

## b) El resultado es la tercer palabra que más apariciones tiene.
FALSO - El resultado son las 3 palabras con mas frecuencia.

## c) El resultado son las 3 palabras más largas.
FALSO - El resultado son las 3 palabras con mas frecuencia.

## d) El flatMap devuelve un registro para cada palabra en el archivo.
FALSO - El flatMap aplica la función a todos los elementos del RDD y luego hace un flat de los resultdos.

In [237]:
archivo = sc.textFile("Data/Archivo.txt")
archivo.collect()

[u'datos spark guia', u'spark hadoop ', u'guia datos spark gradience']